<a href="https://colab.research.google.com/github/mcgmed/Tensorflow/blob/main/Introduction-to-Natural-Language-Processing/Introduction_to_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['Today is a sunny day', 'Today is a rainy day']
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'rainy': 6}


In [ ]:
sentences = ['Today is a sunny day', 'Today is a rainy day', 'Is it sunny today?'] # Be careful about 'today' word with question mark at the last sentence.
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index) # Tokenizer can understand that two words are just same.
# This behavior is controlled by the filters parameter to the tokenizer, which defaults to removing all punctuation except the apostrophe character.

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'rainy': 6, 'it': 7}


In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[1, 2, 3, 5, 4], [1, 2, 3, 6, 4], [2, 7, 5, 1]]


In [ ]:
test_data = ['Today is a snowy day', 'Will it be rainy tomorrow?']

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)
# It converted test_data according to sentences list. It hasn't converted new words.

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'rainy': 6, 'it': 7}
[[1, 2, 3, 4], [7, 6]]


### **Using out-of-vocabulary tokens**

In [ ]:
tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, 'rainy': 7, 'it': 8}
[[2, 3, 4, 1, 6], [1, 8, 1, 7, 1]]


### **Understanding padding**

In [ ]:
sentences = [
'Today is a sunny day',
'Today is a rainy day',
'Is it sunny today?',
'I really enjoyed walking in the snow today'
]

tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
sequences

[[2, 3, 4, 5, 6],
 [2, 3, 4, 7, 6],
 [3, 8, 5, 2],
 [9, 10, 11, 12, 13, 14, 15, 2]]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded = pad_sequences(sequences)
print(padded)

[[ 0  0  0  2  3  4  5  6]
 [ 0  0  0  2  3  4  7  6]
 [ 0  0  0  0  3  8  5  2]
 [ 9 10 11 12 13 14 15  2]]


If you want your sequences to be padded with zeros at the end, you can use:

In [ ]:
padded = pad_sequences(sequences, padding='post')
print(padded)

[[ 2  3  4  5  6  0  0  0]
 [ 2  3  4  7  6  0  0  0]
 [ 3  8  5  2  0  0  0  0]
 [ 9 10 11 12 13 14 15  2]]


What if you don’t want this, perhaps because you have one crazy long sentence that means you’d have too much padding in the padded sequences? To fix that you can use the **maxlen** parameter, specifying the desired maximum length, when calling **pad_sequences**, like this:

In [ ]:
padded = pad_sequences(sequences, padding='post', maxlen=6)
print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [11 12 13 14 15  2]]


Now your padded sequences are all the same length, and there isn’t too much padding. You have lost some words from your longest sentence, though, and they’ve been truncated from the beginning. What if you don’t want to lose the words from the beginning and instead want them truncated from the end of the sentence? You can override the default behavior with the **truncating** parameter, as follows:

In [ ]:
padded = pad_sequences(sequences, padding='post', maxlen=6, truncating='post')
print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [ 9 10 11 12 13 14]]


### **Removing Stopwords and Cleaning Text**

The first is to strip out HTML tags. Fortunately, there’s a library called BeautifulSoup that makes this straightforward. For example, if your sentences contain HTML tags such as >br>, they’ll be removed by this code:

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(sentence)
sentence = soup.get_text()